---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

In [6]:
for G in P1_Graphs:
    print(nx.average_clustering(G))
    print(nx.info(G))

0.03167539146454044
Name: barabasi_albert_graph(1000,2)
Type: Graph
Number of nodes: 1000
Number of edges: 1996
Average degree:   3.9920
0.5642419635919628
Name: watts_strogatz_graph(1000,10,0.05)
Type: Graph
Number of nodes: 1000
Number of edges: 5000
Average degree:  10.0000
0.4018222222222227
Name: watts_strogatz_graph(750,5,0.075)
Type: Graph
Number of nodes: 750
Number of edges: 1500
Average degree:   4.0000
0.03780379975223251
Name: barabasi_albert_graph(750,4)
Type: Graph
Number of nodes: 750
Number of edges: 2984
Average degree:   7.9573
0.0033037037037037037
Name: watts_strogatz_graph(750,4,1)
Type: Graph
Number of nodes: 750
Number of edges: 1500
Average degree:   4.0000


<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [7]:
def graph_identification():
    
    ans = ['SW_H', 'SW_H', 'SW_H', 'SW_H', 'SW_H']
    for i, G in enumerate(P1_Graphs):
        deg = max(list(G.degree().values()))
        if deg > 80:
            ans[i] = 'PA'
        else:
            clus = nx.average_clustering(G)
            if clus > 0.1:
                ans[i] = 'SW_L'
    
    return ans
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [18]:
G = nx.read_gpickle('email_prediction.txt')

x = nx.get_node_attributes(G, 'Department')

In [20]:
x[840]

4

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [16]:
def salary_predictions():
    from sklearn.svm import SVC
    
    df = pd.DataFrame(index=G.nodes())
    
    df['Dept'] = pd.Series(nx.get_node_attributes(G, 'Department'))
        
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['btw'] = pd.Series(nx.betweenness_centrality(G))
    df['close'] = pd.Series(nx.closeness_centrality(G))
    
    df['Mgmt'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    train = df[df.Mgmt.notnull()]
    test = df[df.Mgmt.isnull()]
    
    X_train, y_train = train.loc[:, 'Dept':'close'], train.loc[:, 'Mgmt']
    X_test = test.loc[:, 'Dept':'close']
    
    clf = SVC(C=1, random_state=0, probability=True).fit(X_train, y_train)
    
    pred = clf.predict_proba(X_test)[:,1]
    
    ans = pd.Series(pred, index=test.index)
    
    return ans
salary_predictions()

1       0.099229
2       0.728975
5       0.728644
8       0.108023
14      0.288935
18      0.130858
27      0.207150
30      0.228054
31      0.809348
34      0.094737
37      0.099346
40      0.652554
45      0.099103
54      0.526936
55      0.814646
60      0.567249
62      0.513943
65      0.153673
77      0.099571
79      0.200920
97      0.099421
101     0.098825
103     0.606356
108     0.052141
113     0.246055
122     0.100258
141     0.070586
142     0.949538
144     0.225985
145     0.175873
          ...   
913     0.099296
914     0.099305
915     0.106721
918     0.093388
923     0.097484
926     0.097885
931     0.106899
934     0.069701
939     0.111665
944     0.085639
945     0.092393
947     0.102183
950     0.101045
951     0.097613
953     0.268122
959     0.099152
962     0.093990
963     0.233072
968     0.101684
969     0.098432
974     0.098898
984     0.081148
987     0.111860
989     0.089049
991     0.125811
992     0.089449
994     0.095177
996     0.0991

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [3]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [21]:
def new_connections_predictions():
    
    from sklearn.linear_model import LogisticRegression
    
    df = future_connections
    
    dept = nx.get_node_attributes(G, 'Department')
    
    df['common'] = df.index.map(lambda x:len(list(nx.common_neighbors(G, x[0], x[1]))))
    df['jaccard'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['pref'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    df['dept'] = df.index.map(lambda x:dept[x[0]] == dept[x[1]])
    
    train = df[df['Future Connection'].notnull()]
    test = df[df['Future Connection'].isnull()]
    
    X_train, y_train = train.loc[:, 'common':'dept'], train.loc[:, 'Future Connection']
    X_test = test.loc[:, 'common':'dept']
    
    clf = LogisticRegression(random_state=0).fit(X_train, y_train)
    
    pred = clf.predict_proba(X_test)[:,1]
    
    ans = pd.Series(pred, index=test.index)

    return ans
new_connections_predictions()

(107, 348)    0.026599
(542, 751)    0.009791
(20, 426)     0.613838
(50, 989)     0.009881
(942, 986)    0.009977
(324, 857)    0.009868
(13, 710)     0.148385
(19, 271)     0.125061
(319, 878)    0.009912
(659, 707)    0.009801
(49, 843)     0.009940
(208, 893)    0.009822
(377, 469)    0.006603
(405, 999)    0.017291
(129, 740)    0.015848
(292, 618)    0.016549
(239, 689)    0.009918
(359, 373)    0.007818
(53, 523)     0.271394
(276, 984)    0.009935
(202, 997)    0.009954
(604, 619)    0.039946
(270, 911)    0.009916
(261, 481)    0.067237
(200, 450)    0.980290
(213, 634)    0.009764
(644, 735)    0.035858
(346, 553)    0.009544
(521, 738)    0.009126
(422, 953)    0.016053
                ...   
(672, 848)    0.009916
(28, 127)     0.953173
(202, 661)    0.009506
(54, 195)     0.999840
(295, 864)    0.009854
(814, 936)    0.009778
(839, 874)    0.009977
(139, 843)    0.009862
(461, 544)    0.008839
(68, 487)     0.008789
(622, 932)    0.009833
(504, 936)    0.014864
(479, 528) 